In [ ]:
!python --version

Python 3.7.3


In [ ]:
# Start writing your code here...

In [ ]:
!ls /datasets/data-export

In [ ]:
# imports
import json
from collections import MutableMapping, Counter
from pathlib import Path
from itertools import chain

/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# create directories if not exists
def create_directory(path_):
    path_.mkdir(parents=True, exist_ok=True)

# json reader row by row
def json_reader(json_file, line_parser=None, read_many=None):
    with open(json_file, 'r') as f:
        lines = f.readlines()[:read_many] if read_many else f.readlines()

        for line in lines:
            line = line.strip()
            data = json.loads(line)

            # flatten dictionary
            if line_parser:
                data = line_parser(data)
            yield data

def convert_flatten(d, parent_key ='', sep ='.'): 
    items = [] 
    for k, v in d.items(): 
        new_key = parent_key + sep + k if parent_key else k 
  
        if isinstance(v, MutableMapping): 
            items.extend(convert_flatten(v, new_key, sep = sep).items()) 
        else: 
            items.append((new_key, v)) 
    return dict(items)  

def create_json(data, save_location, new_line=False):
    with open(save_location, 'w') as outfile:
        json.dump(data, outfile, sort_keys=True)

        if new_line:
            outfile.write('\n')

def str_to_json(string):
    j = json.loads(string)
    return j

def pretty_json(json_dict):
    if type(json_dict) == str:
        json_dict = str_to_json(json_dict)
    pretty = json.dumps(json_dict, indent=2, sort_keys=True)
    return pretty


## Only 5 Lines 

In [ ]:
from pathlib import Path

s3_files_loc = Path('/datasets/data-export')
new_files_loc = Path('./raw_data2')

for folder in s3_files_loc.glob('*'):
    folder_name = folder.name

    save_folder = (new_files_loc / folder_name)

    # make directory
    create_directory(save_folder)

    # # loop json files in folder
    for file_ in folder.glob('*.json'):
        file_name = file_.name
        save_name = (save_folder / file_name)

        data = {'data': [line for line in json_reader(file_, convert_flatten, 5)]}
        create_json(data, str(save_name))

## All Lines

In [ ]:
from pathlib import Path

s3_files_loc = Path('/datasets/data-export')
new_files_loc = Path('./raw_data')

for folder in s3_files_loc.glob('*'):
    folder_name = folder.name

    save_folder = (new_files_loc / folder_name)

    # make directory
    create_directory(save_folder)

    # # loop json files in folder
    for file_ in folder.glob('*.json'):
        file_name = file_.name
        save_name = (save_folder / file_name)
        
        data = [line for line in json_reader(file_, convert_flatten)]
        create_json(data, str(save_name))

## Testing Raw

In [ ]:
with open('./raw_data/Conversions/part-00000-tid-4871816523311964264-aeb8c126-56be-4c74-a7ce-de3da53de005-716-1-c000.json', 'r') as f:
    data = json.load(f)
    print(type(data))

<class 'str'>


In [ ]:
df.columns.values

array(['advancePurchaseRangeType', 'browserRequest.browser',
       'browserRequest.browserFamily', 'browserRequest.deviceFamily',
       'browserRequest.os', 'browserRequest.osFamily',
       'browserRequest.referrerUrl', 'browserRequest.screenHeight',
       'browserRequest.screenPixelDepth', 'browserRequest.screenWidth',
       'browserRequest.userAgent', 'carDropoffAirportCode',
       'carPickupAirportCode', 'conversionTransactionType', 'cookieSize',
       'destination', 'entityId', 'isPublisherConversion',
       'knownHotelProperty', 'orderId', 'originalConversionCurrency',
       'origination', 'pageId', 'productCategoryType', 'publisherType',
       'publisherUserId', 'publisherUserIdPerPub', 'request.buildNumber',
       'request.requestId', 'request.requestedAt', 'siteId', 'siteType',
       'thirdPartyCookieAssigned', 'thirdPartyWebuserId', 'travelDateEnd',
       'travelDateStart', 'travelers', 'tripType', 'rooms'], dtype=object)

## Ensure Same Keys Across All

In [ ]:
new_files_loc = Path('./raw_data')

# loop all folders in raw data
# loop all json files in folder
# compile all json data (list of dictionaries) into single iterable
# get all keys from list of dictionaries
# update all dictionaries to have keys that are missing

def get_data(file_):
    with open(file_, 'r') as f:
        data = json.load(f)
        return data

for folder in new_files_loc.glob('*'):
    # produces generator of list of dictionaries
    all_json_data = (get_data(file_) for file_ in folder.glob('*.json'))

    # create single list of all dictionarys (rows) from all json files
    flattened = chain.from_iterable(all_json_data)

    # all keys for folder type
    all_keys = set(chain.from_iterable(flattened))

    # loop through dictionaries and update with missing keys

    # create json
    create_json(data, f'./dataframes/{folder.name}.json')
    

KeyboardInterrupt: 

In [ ]:
file_ = './raw_data/Conversions/part-00000-tid-4871816523311964264-aeb8c126-56be-4c74-a7ce-de3da53de005-716-1-c000.json'

with open(file_, 'r') as f:
    data = json.load(f)
    test = set(chain.from_iterable(data))
    for t in test:
        print(t)

travelDateEnd
request.requestId
orderId
isPublisherConversion
cookieSize
publisherType
thirdPartyCookieAssigned
browserRequest.browserFamily
thirdPartyWebuserId
entityId
browserRequest.userAgent
browserRequest.os
publisherUserIdPerPub
browserRequest.screenPixelDepth
browserRequest.screenWidth
browserRequest.deviceFamily
request.requestedAt
travelDateStart
pageId
browserRequest.osFamily
browserRequest.browser
advancePurchaseRangeType
request.buildNumber
browserRequest.referrerUrl
travelers
siteId
originalConversionCurrency
tripType
conversionTransactionType
origination
rooms
browserRequest.screenHeight
knownHotelProperty
carPickupAirportCode
productCategoryType
siteType
destination
carDropoffAirportCode
publisherUserId


In [ ]:
new_files_loc = Path('./raw_data')

def get_data(file_):
    with open(file_, 'r') as f:
        data = json.load(f)
        return data

def update_dic(dic, full_key_list):
    for k in (full_key_list - dic.keys()):
        dic[k] = None
    return dic

for folder in new_files_loc.glob('*'):
    # produces generator of list of dictionaries
    all_json_data = (get_data(file_) for file_ in folder.glob('*.json'))

    # create single list of all dictionarys (rows) from all json files
    flattened = chain.from_iterable(all_json_data)

    # get all of the keys from flattened
    all_keys = set(chain.from_iterable(flattened))

    # update dictionaries with missing keys
    updated_flattened = (update_dic(dic, all_keys) for dic in flattened)
    
    # loop json lines and create json
    for line in updated_flattened:
        print(line)
        # create_json(line, f'./dataframes/{folder.name}.json', True)
  